# Reclami

## Inputs

In [2]:
from pydantic_settings import BaseSettings, SettingsConfigDict

ENV_FILE = ".env"

class PersonalData(BaseSettings):

    model_config = SettingsConfigDict(env_file=ENV_FILE)

    user_surname: str
    user_name: str
    gender: str
    birth_date: str
    birth_nation: str
    comune_nascita: str
    codice_fiscale: str
    address: str
    house_number: str
    cap: str
    nation: str
    provincia: str
    comune: str
    location: str
    email: str
    phone_number: str
    enable_phone_number_to_be_contacted: bool 

pd = PersonalData()

# Dati comunicazione
service_type = ""  # Servizio bus
tipologia = ""  # Servizio
categoria = "" # [Corsa non effettuata, Frequenza o orario non rispettato]
bacino = ""  # Bologna
linea = ""  # 20
direzione = ""  # Pilastro Rotonda Malaguti
matricola_veicolo = ""
matricola_conducente = ""
data = ""  # 11/11/2024
ora = ""  # 09:11
luogo = ""  # fermata N xxxx
testo = """"""  # testo del reclamo

enable_indennizzo = True  # Try always

# Riepilogo
privacy = True

In [1]:
from playwright.async_api import async_playwright
import time

url = "https://solweb.tper.it/web/communications/complaints.aspx"

playwright = await async_playwright().start()

browser = await playwright.chromium.launch(headless=False)
page = await browser.new_page()

await page.goto(url)

<Response url='https://solweb.tper.it/web/communications/complaints.aspx' request=<Request url='https://solweb.tper.it/web/communications/complaints.aspx' method='GET'>>

# Rifiuta cookie non necessari

In [ ]:
cookie_class = "a.c-cookie-banner-header-refuse-btn"
await page.click(cookie_class, timeout=1000)
time.sleep(0.5)

# Dati anagrafici

In [3]:
# text fields
surname_id = '#ctl00_ContentPlaceHolderMain_txtCognome'
name_id = "#ctl00_ContentPlaceHolderMain_txtNome"
birth_date_id = "#ctl00_ContentPlaceHolderMain_txtDataNascita_dateInput"
birth_nation_id = "#ctl00_ContentPlaceHolderMain_cboNazioneNascita_Input"
comune_nascita_id = "#ctl00_ContentPlaceHolderMain_cboComuneNascita_Input"
codice_fiscale_id = "#ctl00_ContentPlaceHolderMain_txtCodiceFiscale"
address_id = "#ctl00_ContentPlaceHolderMain_txtIndirizzo"
house_number_id = "#ctl00_ContentPlaceHolderMain_txtNumeroCivico"
cap_id = "#ctl00_ContentPlaceHolderMain_txtCap"
nation_id = "#ctl00_ContentPlaceHolderMain_cboNazione_Input"
provincia_id = "#ctl00_ContentPlaceHolderMain_cboProvincia_Input"
comune_id = "#ctl00_ContentPlaceHolderMain_cboComune_Input"
location_id = "#ctl00_ContentPlaceHolderMain_txtLocalita"
email_id = "#ctl00_ContentPlaceHolderMain_txtEmail"
phone_number_id = "#ctl00_ContentPlaceHolderMain_txtTelefonoCellulare"

text_fields = [
    (surname_id, pd.user_surname),
    (name_id, pd.user_name),
    (birth_date_id, pd.birth_date),
    (birth_nation_id, pd.birth_nation),
    (comune_nascita_id, pd.comune_nascita),
    (codice_fiscale_id, pd.codice_fiscale),
    (address_id, pd.address),
    (house_number_id, pd.house_number),
    (cap_id, pd.cap),
    (nation_id, pd.nation),
    (provincia_id, pd.provincia),
    (comune_id, pd.comune),
    (location_id, pd.location),
    (email_id, pd.email),
    (phone_number_id, pd.phone_number),
]

for tf in text_fields:
    await page.fill(tf[0], tf[1])
    time.sleep(0.5)
    await page.click(selector="body", position={'x': 0, 'y': page.viewport_size['height']})
    time.sleep(0.5)

In [4]:
# radio buttons
# subject_type_id = "#ContentPlaceHolderMain_rbtlstTipoSoggetto_0"
gender_id = "#ContentPlaceHolderMain_rbtlstSesso_0"

if pd.gender == "M":
    gender_id = gender_id[:-1] + "1"

await page.check(gender_id)

In [5]:
# check buttons
enable_phone_number_to_be_contacted_id = "#ContentPlaceHolderMain_chkModalitaRispostaTelefonica"

if pd.enable_phone_number_to_be_contacted:
    await page.check(enable_phone_number_to_be_contacted_id)
else:
    await page.uncheck(enable_phone_number_to_be_contacted_id)

In [6]:
# submit button
submit_button_id = "#ctl00_ContentPlaceHolderMain_btAvanti1_input"

await page.click(submit_button_id)
time.sleep(0.5)

# Dati comunicazione

In [7]:
async def get_li_pos(
    dropdown_id: str,
    input_text: str
) -> int:
    li = page.locator(dropdown_id)
    time.sleep(0.5)
    handles = await li.locator('ul > li').element_handles()

    for idx, h in enumerate(handles):
        t = await h.inner_text()
        if t.strip().lower() == input_text.lower():
            return idx+1
        
    raise Exception("choice '{}' not found!".format(input_text))

In [8]:
# Dropdown
service_type_id = "#ctl00_ContentPlaceHolderMain_ddlstTipoServizio"
service_type_dropdown_id = "#ctl00_ContentPlaceHolderMain_ddlstTipoServizio_DropDown"
tipologia_id = "#ctl00_ContentPlaceHolderMain_ddlstTipologiaComunicazione"
tipologia_dropdown_id = "#ctl00_ContentPlaceHolderMain_ddlstTipologiaComunicazione_DropDown"
categoria_id = "#ctl00_ContentPlaceHolderMain_ddlstCategoriaComunicazione"
categoria_dropdown_id = "#ctl00_ContentPlaceHolderMain_ddlstCategoriaComunicazione_DropDown"
bacino_id = "#ctl00_ContentPlaceHolderMain_ddlstBacinoAutomobilistico"
bacino_dropdown_id = "#ctl00_ContentPlaceHolderMain_ddlstBacinoAutomobilistico_DropDown"
linea_id = "#ctl00_ContentPlaceHolderMain_ddlstLineaAutomobilistica"
linea_dropdown_id = "#ctl00_ContentPlaceHolderMain_ddlstLineaAutomobilistica_DropDown"

dropdown = [
    (service_type_id, service_type_dropdown_id, service_type),
    (tipologia_id, tipologia_dropdown_id, tipologia),
    (categoria_id, categoria_dropdown_id, categoria),
    (bacino_id, bacino_dropdown_id, bacino),
    (linea_id, linea_dropdown_id, linea)
]

for d in dropdown:
    await page.click(d[0])
    time.sleep(0.5)
    li = (page
        .locator(d[1])
        .locator('ul > li:nth-child({})'.format(await get_li_pos(d[1], d[2]))))
    time.sleep(0.5)
    await li.click(timeout=10000)

In [9]:
# text fields
direzione_id = "#ctl00_ContentPlaceHolderMain_txtDirezioneVeicolo"
matricola_veicolo_id = "#ctl00_ContentPlaceHolderMain_txtMatricolaVeicolo"
matricola_conducente_id = "#ctl00_ContentPlaceHolderMain_txtMatricolaConducente"
data_id = "#ctl00_ContentPlaceHolderMain_txtDataEvento_dateInput"
ora_id = "#ctl00_ContentPlaceHolderMain_txtOraEvento_dateInput"
luogo_id = "#ctl00_ContentPlaceHolderMain_txtLuogoEvento"
testo_id = "#ctl00_ContentPlaceHolderMain_txtTesto"

text_fields = [
    (direzione_id, direzione),
    (matricola_veicolo_id, matricola_veicolo),
    (matricola_conducente_id, matricola_conducente),
    (data_id, data),
    (ora_id, ora),
    (luogo_id, luogo),
    (testo_id, testo),
]

for tf in text_fields:
    await page.fill(tf[0], tf[1])
    time.sleep(0.5)
    await page.click(selector="body", position={'x': 0, 'y': page.viewport_size['height']})
    time.sleep(0.5)

In [10]:
# check buttons
enable_indennizzo_id = "#ContentPlaceHolderMain_chkRimborso"

if enable_indennizzo:
    await page.check(enable_indennizzo_id)
else:
    await page.uncheck(enable_indennizzo_id)

In [11]:
# submit button
submit_button_id = "#ctl00_ContentPlaceHolderMain_btAvanti2_input"

await page.click(submit_button_id)
time.sleep(0.5)

# Riepilogo

In [12]:
# check buttons
privacy_id = "#ContentPlaceHolderMain_chkConsensoPrivacy"
await page.check(privacy_id)

In [13]:
# submit button
submit_button_id = "#ctl00_ContentPlaceHolderMain_btAvanti3_input"

await page.click(submit_button_id)
time.sleep(0.5)

# Close

In [14]:
await browser.close()
await playwright.stop()